In [61]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix


In [62]:
data = pd.read_csv('WMT1.csv')
data_17 = data[data.Year == 2017]

# calculate the mean std
mean_17 = data_17.groupby('Week_Number')['Return'].mean().tolist()
std_17 = data_17.groupby('Week_Number')['Return'].std().tolist()
week_id_17 = list(data_17.groupby('Week_Number').indices.keys())
color_17 = data_17.groupby('Week_Number')['label'].agg(lambda x: x.mode()).tolist()

data_18 = data[data.Year == 2018]
data_18.drop(data_18.index[-1], inplace=True)

# calculate the mean std
mean_18 = data_18.groupby('Week_Number')['Return'].mean().tolist()
std_18 = data_18.groupby('Week_Number')['Return'].std().tolist()
weekly_18 = data[(data.Year == 2018) & (data.Weekday == 'Friday')]['Adj Close'].values  # select Friday's data

week_id_18 = list(data_18.groupby('Week_Number').indices.keys())
color_18 = data_18.groupby('Week_Number')['label'].agg(lambda x: x.mode()).tolist()


In [63]:
df_17 = pd.DataFrame(
    {
        'id':week_id_17,
        'mean':mean_17,
        'std':std_17,
        'label':color_17
    },
    columns = ['id','mean','std','label']
)

df_18 = pd.DataFrame(
    {
        'id':week_id_18,
        'mean':mean_18,
        'std':std_18,
        'label':color_18
    },
    columns = ['id','mean','std','label']
)

In [64]:
x_train = df_17[['mean','std']].values
y_train= df_17['label'].values

x_test = df_18[['mean','std']].values
y_test = df_18['label'].values

In [65]:
NB_classifier = GaussianNB().fit(x,y)
prediction = NB_classifier.predict(x_test)

acc = 1 - np.mean(prediction != y_test)
acc # qusetion 1

0.8269230769230769

In [66]:
a = confusion_matrix(y_test, prediction)
a # question 2

array([[29,  2],
       [ 7, 14]])

In [67]:
TPR = a[0][0]/(a[0][0]+a[0][1])
TNR = a[1][1]/(a[1][0]+a[1][1])
print('true positive rate is :',TPR)
print('true negative rate is :',TNR)


true positive rate is : 0.9354838709677419
true negative rate is : 0.6666666666666666


In [68]:
previous_shares = 0
previous_balance = 100
weekly_balance = []
for i in range(len(weekly_18)-1):
    if prediction[i+1] == 'green':
        if previous_shares ==0 :
            previous_shares = previous_balance/weekly_18[i]
    elif prediction[i+1] == 'red':
        if previous_shares != 0:
            previous_balance = previous_shares*weekly_18[i]
            previous_shares = 0

    weekly_balance.append(previous_balance)
result1 = weekly_balance[-1]

In [69]:
result2 = 100/weekly_18[0]*weekly_18[-1]

print('result based on my labels: %f $'%result1)
print('result based on buy and hold: %f $'%result2)
print('buy and hold strategy results in a large amount at the end of the year')

result based on my labels: 93.111329 $
result based on buy and hold: 94.188504 $
buy and hold strategy results in a large amount at the end of the year
